In [1]:
from PIL import Image

In [2]:
import numpy as np 
import os
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
import cv2

C:\Users\Nahian\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from osgeo import gdal
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
def unet_1(pretrained_weights = None, input_size = (256,256,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = merge([drop4,up6], mode = 'concat', concat_axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = merge([conv3,up7], mode = 'concat', concat_axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = merge([conv2,up8], mode = 'concat', concat_axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = merge([conv1,up9], mode = 'concat', concat_axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'softmax')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

In [5]:
def unet_2(input_size = (256,256,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])
    
    model.compile(optimizer=Adam(lr=1e-5), loss= 'binary_crossentropy', metrics= ['accuracy'])

    return model

In [6]:
model = unet_2()

In [7]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 32) 9248        conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 128, 128, 32) 0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (

In [8]:
train_imageDir = "train\\hillshade"

In [9]:
train_labelDir = "train\\road"

In [10]:
test_imageDir = "test\\hillshade"

In [11]:
def images_to_array(imageDir):
    image_list = os.listdir(imageDir)
    arrayList = []
    for ind, image_name in enumerate(image_list):
        img_path = os.path.join(imageDir,image_name)

        img = cv2.imread(img_path, 0)
        resized_img = cv2.resize(img, (256, 256))
        
        resized_img = np.reshape(resized_img, resized_img.shape + (1,))
        
        arrayList.append(resized_img)
    
    arrayList = np.array(arrayList)
    return arrayList

In [12]:
def load_data(directory):
    image_list = os.listdir(directory)
    arrayList = []
    for ind, image_name in enumerate(image_list):
        img_path = os.path.join(directory,image_name)

        ds = gdal.Open(img_path)
        img = ds.ReadAsArray()
        img = cv2.resize(img, (256, 256))
        print(img.shape)
        resized_img = np.reshape(img, img.shape + (1,))
        
        arrayList.append(resized_img)
    
    arrayList = np.array(arrayList)
    return arrayList

In [13]:
def load_label(directory):
    image_list = os.listdir(directory)
    arrayList = []
    for ind, image_name in enumerate(image_list):
        img_path = os.path.join(directory,image_name)

        ds = gdal.Open(img_path)
        img = ds.ReadAsArray()
        img = cv2.resize(img, (256, 256))
        ## preprocess
        for row in range(img.shape[0]):
            for col in range(img.shape[1]):
                if img[row][col] > 0:
                    img[row][col] = 1
        print(img.shape)
        resized_img = np.reshape(img, img.shape + (1,))
        
        arrayList.append(resized_img)
    
    arrayList = np.array(arrayList)
    return arrayList

In [14]:
train_x = load_data(train_imageDir)

(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)


In [15]:
train_y = load_label(train_labelDir)

(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)
(256, 256)


In [16]:
train_x.shape

(40, 256, 256, 1)

In [17]:
train_y.shape

(40, 256, 256, 1)

In [18]:
train_x

array([[[[241],
         [234],
         [232],
         ...,
         [197],
         [197],
         [226]],

        [[223],
         [194],
         [189],
         ...,
         [134],
         [138],
         [197]],

        [[229],
         [195],
         [184],
         ...,
         [145],
         [146],
         [199]],

        ...,

        [[227],
         [194],
         [199],
         ...,
         [ 77],
         [ 76],
         [174]],

        [[229],
         [198],
         [197],
         ...,
         [ 70],
         [ 92],
         [176]],

        [[242],
         [229],
         [231],
         ...,
         [169],
         [177],
         [215]]],


       [[[255],
         [255],
         [255],
         ...,
         [177],
         [174],
         [222]],

        [[255],
         [255],
         [255],
         ...,
         [ 96],
         [ 98],
         [186]],

        [[255],
         [255],
         [255],
         ...,
         [101],
         [

In [19]:
train_x = train_x.astype('float32')
mean = np.mean(train_x)  # mean for data centering
std = np.std(train_x)

In [20]:
#train_x -= mean
#train_x /= std

In [21]:
train_y = train_y.astype('float32')
#train_y *= 255. # scale masks to [0, 1]

In [22]:
model_checkpoint = ModelCheckpoint('road_detection_hillshade.hdf5', monitor='loss',verbose=1, save_best_only=True)

In [23]:
model.fit(train_x, train_y, batch_size=5, epochs=100, verbose = 1, callbacks=[model_checkpoint])

Epoch 1/100
40/40 [==============================] - 14s 353ms/step - loss: 2.9354 - acc: 0.0519

Epoch 00001: loss improved from inf to 2.93542, saving model to road_detection_hillshade.hdf5
Epoch 2/100
40/40 [==============================] - 3s 84ms/step - loss: 1.9682 - acc: 0.1060

Epoch 00002: loss improved from 2.93542 to 1.96824, saving model to road_detection_hillshade.hdf5
Epoch 3/100
40/40 [==============================] - 3s 84ms/step - loss: 1.1926 - acc: 0.3068

Epoch 00003: loss improved from 1.96824 to 1.19263, saving model to road_detection_hillshade.hdf5
Epoch 4/100
40/40 [==============================] - 3s 85ms/step - loss: 0.7100 - acc: 0.6442

Epoch 00004: loss improved from 1.19263 to 0.70998, saving model to road_detection_hillshade.hdf5
Epoch 5/100
40/40 [==============================] - 3s 85ms/step - loss: 0.4646 - acc: 0.8336

Epoch 00005: loss improved from 0.70998 to 0.46457, saving model to road_detection_hillshade.hdf5
Epoch 6/100
40/40 [=============

40/40 [==============================] - 3s 85ms/step - loss: 0.1049 - acc: 0.9798

Epoch 00043: loss did not improve
Epoch 44/100
40/40 [==============================] - 3s 85ms/step - loss: 0.1041 - acc: 0.9800

Epoch 00044: loss improved from 0.10487 to 0.10414, saving model to road_detection_hillshade.hdf5
Epoch 45/100
40/40 [==============================] - 3s 84ms/step - loss: 0.1039 - acc: 0.9799

Epoch 00045: loss improved from 0.10414 to 0.10388, saving model to road_detection_hillshade.hdf5
Epoch 46/100
40/40 [==============================] - 3s 85ms/step - loss: 0.1026 - acc: 0.9800

Epoch 00046: loss improved from 0.10388 to 0.10262, saving model to road_detection_hillshade.hdf5
Epoch 47/100
40/40 [==============================] - 3s 84ms/step - loss: 0.1032 - acc: 0.9800

Epoch 00047: loss did not improve
Epoch 48/100
40/40 [==============================] - 3s 84ms/step - loss: 0.1027 - acc: 0.9799

Epoch 00048: loss did not improve
Epoch 49/100
40/40 [===============

40/40 [==============================] - 3s 85ms/step - loss: 0.0946 - acc: 0.9801

Epoch 00092: loss did not improve
Epoch 93/100
40/40 [==============================] - 3s 84ms/step - loss: 0.0939 - acc: 0.9802

Epoch 00093: loss improved from 0.09405 to 0.09393, saving model to road_detection_hillshade.hdf5
Epoch 94/100
40/40 [==============================] - 3s 85ms/step - loss: 0.0940 - acc: 0.9801

Epoch 00094: loss did not improve
Epoch 95/100
40/40 [==============================] - 3s 85ms/step - loss: 0.0932 - acc: 0.9802

Epoch 00095: loss improved from 0.09393 to 0.09322, saving model to road_detection_hillshade.hdf5
Epoch 96/100
40/40 [==============================] - 3s 85ms/step - loss: 0.0930 - acc: 0.9802

Epoch 00096: loss improved from 0.09322 to 0.09296, saving model to road_detection_hillshade.hdf5
Epoch 97/100
40/40 [==============================] - 3s 85ms/step - loss: 0.0932 - acc: 0.9802

Epoch 00097: loss did not improve
Epoch 98/100
40/40 [===============

In [24]:
test_x = load_data(test_imageDir)

(256, 256)
(256, 256)
(256, 256)
(256, 256)


In [25]:
test_x = test_x.astype('float32')
mean = np.mean(test_x)  # mean for data centering
std = np.std(test_x)
#test_x -= mean
#test_x /= std

In [26]:
test_x.shape[0]

4

In [15]:
model = unet_2()
model.load_weights("unet_membrane.hdf5")


In [28]:
for i in range(test_x.shape[0]):
    resutls = model.predict(test_x[i:i+1])
    output_img = np.reshape(resutls[0], (256, -1))
    output_img = cv2.resize(output_img, (500, 500))
    cv2.imshow('image', output_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [31]:
resutls = model.predict(test_x[2:3])

In [32]:
resutls.shape

(1, 256, 256, 1)

In [33]:
output_img = np.reshape(resutls[0], (256, -1))

In [34]:
output_img.shape

(256, 256)

In [35]:
output_img

array([[4.3252644e-01, 2.4907096e-01, 3.0615741e-01, ..., 4.9177814e-01,
        2.3634380e-01, 7.7649608e-02],
       [6.0297749e-03, 1.3119867e-01, 1.0613156e-03, ..., 5.9951097e-04,
        3.6306159e-05, 5.8183126e-04],
       [1.4476083e-01, 1.4969393e-02, 8.3647981e-02, ..., 2.1332140e-01,
        2.3087619e-02, 1.2244937e-02],
       ...,
       [3.3219457e-02, 1.3741260e-02, 6.1034165e-02, ..., 4.1503999e-03,
        1.6697241e-02, 4.4134185e-03],
       [8.9497544e-04, 5.1374175e-02, 5.5829850e-03, ..., 1.5406779e-02,
        2.0357051e-03, 3.0623931e-01],
       [2.3456940e-03, 6.8830922e-03, 1.2551156e-03, ..., 2.4082260e-01,
        2.8199579e-03, 2.9383150e-01]], dtype=float32)

In [36]:
for row in range(output_img.shape[0]):
        for col in range(output_img.shape[1]):
            if output_img[row][col] > 0.05:
                output_img[row][col] = 1
            else:
                output_img[row][col] = 0

In [37]:
cv2.imshow('image', output_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [40]:
np.min(output_img)

8.946197e-08

In [28]:
def labelVisualize(num_class,color_dict,img):
    img = img[:,:,0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i,:] = color_dict[i]
    return img_out / 255


In [23]:
def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
    io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)

In [34]:
output_img = output_img*255

In [48]:
type(output_img)

numpy.ndarray

In [17]:
test_img = train_y[0]

In [56]:
for row in range(256):
    for col in range(256):
        if output_img[row][col] < 0.7:
            print(row, col)
            output_img[row][col] = 0
        else:
            output_img[row][col] = 1

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 32
0 33
0 34
0 35
0 36
0 37
0 38
0 39
0 40
0 41
0 42
0 43
0 44
0 45
0 46
0 47
0 48
0 49
0 50
0 51
0 52
0 53
0 54
0 55
0 56
0 57
0 58
0 59
0 60
0 61
0 62
0 63
0 64
0 65
0 66
0 67
0 68
0 69
0 70
0 71
0 72
0 73
0 74
0 75
0 76
0 77
0 78
0 79
0 80
0 81
0 82
0 83
0 84
0 85
0 86
0 87
0 88
0 89
0 90
0 91
0 92
0 93
0 94
0 95
0 96
0 97
0 98
0 99
0 100
0 101
0 102
0 103
0 104
0 105
0 106
0 107
0 108
0 109
0 110
0 111
0 112
0 113
0 114
0 115
0 116
0 117
0 118
0 119
0 120
0 121
0 122
0 123
0 124
0 125
0 126
0 127
0 128
0 129
0 130
0 131
0 132
0 133
0 134
0 135
0 136
0 137
0 138
0 139
0 140
0 141
0 142
0 143
0 144
0 145
0 146
0 147
0 148
0 149
0 150
0 151
0 152
0 153
0 154
0 155
0 156
0 157
0 158
0 159
0 160
0 161
0 162
0 163
0 164
0 165
0 166
0 167
0 168
0 169
0 170
0 171
0 172
0 173
0 174
0 175
0 176
0 177
0 178
0 179
0 180
0 181
0 182
0 183
0 184


215 2
215 3
215 4
215 248
215 250
215 251
215 252
215 253
215 254
215 255
216 0
216 1
216 2
216 3
216 4
216 246
216 247
216 248
216 249
216 250
216 251
216 252
216 253
216 254
216 255
217 0
217 1
217 2
217 3
217 248
217 249
217 250
217 251
217 252
217 253
217 254
217 255
218 0
218 1
218 2
218 3
218 240
218 246
218 248
218 250
218 251
218 252
218 253
218 254
218 255
219 0
219 1
219 2
219 240
219 251
219 252
219 253
219 254
219 255
220 0
220 1
220 238
220 242
220 250
220 251
220 252
220 253
220 254
220 255
221 0
221 252
221 253
221 254
221 255
222 0
222 1
222 254
222 255
223 0
223 1
223 2
223 254
223 255
224 0
224 1
224 253
224 254
224 255
225 0
225 253
225 254
225 255
226 0
226 1
226 254
226 255
227 0
227 1
227 2
227 254
227 255
228 0
228 1
228 253
228 254
228 255
229 0
229 1
229 2
229 253
229 255
230 0
230 1
230 2
230 252
230 253
230 254
230 255
231 0
231 1
231 2
231 255
232 0
232 1
232 2
232 3
232 254
232 255
233 0
233 1
233 2
233 3
233 4
233 253
233 255
234 0
234 1
234 2
234 3
234 25

In [34]:
np.mean(output_img)

0.85251397

In [35]:
np.std(output_img)

0.06989891

In [36]:
output_img = output_img.astype('uint8')


In [37]:
output_img

array([[ 60,  54,  34, ...,  38,  55, 114],
       [ 37,  46,  13, ...,  33,  42,  67],
       [ 31,  30,   7, ...,  15,  25,  57],
       ...,
       [ 60,  40,  26, ...,   5,  13,  25],
       [ 79,  35,  36, ...,  13,  15,  21],
       [ 85,  73,  48, ...,  21,  26,  59]], dtype=uint8)

In [21]:
testImg = train_y[0]*255

In [34]:
cv2.imshow('image', output_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [19]:
for i in range(255):
    print(train_y[0][i])

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.